In [ ]:
%load_ext autoreload
%autoreload 2

import torch
from resnet_module import ResNetModel
import wandb

from resnet_training import get_predictions, plot_confusion_matrix, calculate_gender_bias, calculate_age_bias

In [ ]:
debiasing_runs = ["folm5wbd", "00riantl", "uiznsjjj", "30y8q6vf", "hi6ej7fl", "nl10rh1v", "xfh2oa3v", "i8zwc0lj", "062eb1li", "79uu4gax"]
no_debiasing_runs = ["kzozqfbw", "kdt4bf3x", "k7w8ngzr", "w572bz9r", "9e0j4szk", "3x8qz40u", "n3w92l9t", "ylc3xokv", "w2fevh40", "5guuqk3t"]

In [ ]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

for run in debiasing_runs:
    checkpoint_dir = f'./bias-skin-lesion-detection/{run}/checkpoints/'
    file_name = os.listdir(directory)[0]
    model_path = os.join(checkpoint_dir, file_name)
    
    model = ResNetModel.load_from_checkpoint(model_path, map_location=device)
    predictions, all_labels = get_predictions(model)
    
    metric = trochmetrics.classification.BinaryAccuracy()
    
    calculate_gender_bias(predictions, all_labels)

In [ ]:
# Plot confusion matrix

plot_confusion_matrix(predictions, all_labels)

In [ ]:
# Evaluate bias based on sex
calculate_gender_bias(predictions, all_labels)

In [ ]:
# Evaluate bias based on age groups

calculate_age_bias(predictions, all_labels)